In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/885303/train_dataset_train.csv
!wget https://lodmedia.hb.bizmrg.com/case_files/885303/test_dataset_test.csv

In [ ]:
!pip install catboost

In [ ]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor,NearestCentroid
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold,StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics.pairwise import paired_manhattan_distances
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer
import catboost

In [ ]:
train=pd.read_csv('/content/content/train_dataset_train.csv')
test=pd.read_csv('/content/content/test_dataset_test.csv')

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(train[['index_oper',
 'price_mfi',
 'total_qty_oper_login_0',
 'total_qty_oper_login_1',
 'total_qty_over_index',
 'total_qty_over_index_and_type',
 'transport_pay',
 'weight',
 'weight_mfi',
 'oper_type + oper_attr', 'type', 'priority',
       'is_privatecategory', 'class', 'is_in_yandex', 'is_return', 
       'mailtype', 'mailctg', 'mailrank', 'directctg', 
       'postmark', 'dist_qty_oper_login_1', 'is_wrong_sndr_name', 'is_wrong_rcpn_name',
       'is_wrong_phone_number', 'is_wrong_address']],
                                             train['label'].values,
                                             test_size=0.1,
                                             random_state=0,
                                             stratify=train['label'])

In [ ]:
cls=catboost.CatBoostClassifier(iterations=1000, 
                                learning_rate=0.1,
                                loss_function='Logloss',
                                auto_class_weights='Balanced', 
                                use_best_model=True,
                                task_type='GPU',
                                eval_metric='Logloss',
                                border_count=254,
                                bagging_temperature=0.1)

In [ ]:
cls.fit(X_train,y_train,
        cat_features=['oper_type + oper_attr','type','is_privatecategory', 
                      'is_in_yandex', 'is_return'],
        eval_set=(X_val,y_val),
        verbose=10)

In [ ]:
test['label']=cls.predict(test[['index_oper',
 'price_mfi',
 'total_qty_oper_login_0',
 'total_qty_oper_login_1',
 'total_qty_over_index',
 'total_qty_over_index_and_type',
 'transport_pay',
 'weight',
 'weight_mfi',
 'oper_type + oper_attr', 'type', 'priority',
       'is_privatecategory', 'class', 'is_in_yandex', 'is_return', 
       'mailtype', 'mailctg', 'mailrank', 'directctg', 
       'postmark', 'dist_qty_oper_login_1', 'is_wrong_sndr_name', 'is_wrong_rcpn_name',
       'is_wrong_phone_number', 'is_wrong_address']])

In [ ]:
submit=test[['id','label']]

In [ ]:
submit.to_csv('submitr32.csv',index=False)